In [2]:
import os
from google.colab import drive

print("🔌 Drive Bağlanıyor...")
drive.mount('/content/drive')

print("✅ Ortam Hazır!")

🔌 Drive Bağlanıyor...
Mounted at /content/drive
✅ Ortam Hazır!


In [ ]:
import os
import glob
import re
import gc
import sys
from google.colab import drive

# --- 1. KURTARMA MODU: Kütüphaneler ve Drive ---
print("🚑 KURTARMA MODU BAŞLATILIYOR...")

# Drive Bağlantısı
if not os.path.exists('/content/drive'):
    print("🔌 Drive tekrar bağlanıyor...")
    drive.mount('/content/drive')

# Kütüphane Kontrolü (Silindiyse tekrar kur)
try:
    import langchain
    import chromadb
    print("✅ Kütüphaneler yerinde duruyor.")
except ImportError:
    print("⚠️ Kütüphaneler silinmiş! Tekrar kuruluyor (1-2 dk sürer)...")
    !pip install langchain langchain-community langchain-core langchain-huggingface langchain-chroma chromadb pypdf sentence-transformers -q
    print("✅ Kurulum tamamlandı.")

# Importlar (Kurulumdan sonra yapılmalı)
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# --- AYARLAR ---
PDF_KLASORU = "/content/drive/MyDrive/hukuk-bot/data/ham_veri"
VECTOR_DB_PATH = "/content/drive/MyDrive/hukuk-bot/data/vector_db"

print(f"\n📂 Hedef Klasör: {PDF_KLASORU}")
print(f"💾 Veritabanı Yolu: {VECTOR_DB_PATH}")

# --- 2. AKILLI DEVAM SİSTEMİ ---

# Embedding Modelini Yükle
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Veritabanını Diskten Oku
db = Chroma(
    persist_directory=VECTOR_DB_PATH,
    embedding_function=embedding_model
)

# Zaten İşlenmiş Dosyaları Bul
print("🔍 Veritabanı taranıyor: Hangi dosyalar bitmiş?")
processed_files = set()
try:
    # Veritabanındaki tüm kayıtların metadata'sını çek
    existing_data = db.get()
    if existing_data and 'metadatas' in existing_data:
        for meta in existing_data['metadatas']:
            if meta and 'source' in meta:
                processed_files.add(meta['source'])
    print(f"✅ Tespit edildi: {len(processed_files)} adet kitap ZATEN KAYITLI.")
except Exception as e:
    print(f"⚠️ Veritabanı okuma uyarısı: {e} (Boş olabilir)")

# --- 3. İŞLEME DÖNGÜSÜ ---
pdf_dosyalari = glob.glob(os.path.join(PDF_KLASORU, "**", "*.pdf"), recursive=True)
print(f"📚 Toplam PDF Sayısı: {len(pdf_dosyalari)}")
kalan_sayisi = len(pdf_dosyalari) - len(processed_files)
print(f"▶️ İşlenecek Kalan Dosya: {kalan_sayisi}")

if kalan_sayisi == 0:
    print("\n🎉 MÜJDE! Tüm dosyalar zaten işlenmiş. Hiçbir şey yapmana gerek yok.")
else:
    for i, pdf_path in enumerate(pdf_dosyalari):
        dosya_adi = os.path.basename(pdf_path)

        # KONTROL: Eğer listede varsa PAS GEÇ
        if dosya_adi in processed_files:
            # print(f"⏭️ Atlandı: {dosya_adi}") # Ekranı doldurmasın diye kapalı
            continue

        try:
            print(f"\n📖 [{i+1}/{len(pdf_dosyalari)}] İşleniyor: {dosya_adi}...")
            loader = PyPDFLoader(pdf_path)
            pages = loader.load()
            full_text = "\n".join([p.page_content for p in pages])

            # Regex ile Madde Madde Böl
            pattern = r"(Madde\s+\d+\s*[-–])"
            chunks = re.split(pattern, full_text)

            semantic_chunks = []
            if len(chunks) > 0 and len(chunks[0].strip()) > 10:
                 semantic_chunks.append(chunks[0].strip())

            for k in range(1, len(chunks), 2):
                if k + 1 < len(chunks):
                    baslik = chunks[k]
                    icerik = chunks[k+1]
                    tam_madde = f"{baslik} {icerik}".strip()
                    if len(tam_madde) > 20:
                        semantic_chunks.append(tam_madde)

            # Belge oluştur
            new_docs = []
            for chunk in semantic_chunks:
                new_docs.append(Document(page_content=chunk, metadata={"source": dosya_adi}))

            # KAYDET (Her dosya bitince kaydeder, elektrik gitse de sorun olmaz)
            if len(new_docs) > 0:
                db.add_documents(new_docs)
                print(f"   💾 KAYDEDİLDİ: {len(new_docs)} madde eklendi.")
            else:
                print("   ⚠️ Bu dosyadan madde çıkarılamadı.")

        except Exception as e:
            print(f"   ❌ Hata: {e}")

        # RAM Temizliği
        del full_text, chunks, semantic_chunks, new_docs
        gc.collect()

    print("\n🎉 OPERASYON TAMAMLANDI! Tüm eksikler giderildi.")

🚑 KURTARMA MODU BAŞLATILIYOR...
✅ Kütüphaneler yerinde duruyor.

📂 Hedef Klasör: /content/drive/MyDrive/hukuk-bot/data/ham_veri
💾 Veritabanı Yolu: /content/drive/MyDrive/hukuk-bot/data/vector_db


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔍 Veritabanı taranıyor: Hangi dosyalar bitmiş?
✅ Tespit edildi: 3 adet kitap ZATEN KAYITLI.
📚 Toplam PDF Sayısı: 73
▶️ İşlenecek Kalan Dosya: 70

📖 [1/73] İşleniyor: 1.5.4721_removed2.pdf...
   💾 KAYDEDİLDİ: 1021 madde eklendi.

📖 [2/73] İşleniyor: 1.5.5237_removed.pdf...
   💾 KAYDEDİLDİ: 342 madde eklendi.

📖 [3/73] İşleniyor: AILE HUKUKU (2).pdf...
   💾 KAYDEDİLDİ: 1 madde eklendi.

📖 [4/73] İşleniyor: Aile hukuku Mustafa dural2.pdf...
   ⚠️ Bu dosyadan madde çıkarılamadı.

📖 [5/73] İşleniyor: Aile hukuku Mustafa dural_removed4.pdf...
   ⚠️ Bu dosyadan madde çıkarılamadı.

📖 [6/73] İşleniyor: Aile hukuku notu.pdf...
   💾 KAYDEDİLDİ: 1 madde eklendi.

📖 [7/73] İşleniyor: Anayasa Hukuku Themis 2020_removed2.pdf...
   💾 KAYDEDİLDİ: 1 madde eklendi.

📖 [8/73] İşleniyor: BORCLAR GENEL OGUZMAN2.pdf...
   💾 KAYDEDİLDİ: 2 madde eklendi.

📖 [9/73] İşleniyor: Borlar zel Hukuku.pdf...
   💾 KAYDEDİLDİ: 1 madde eklendi.

📖 [10/73] İşleniyor: CEZA HUKUKU ZEL HKMLER ARTUK 2.pdf...
   💾 KAYDEDİLDİ: 

In [ ]:
# --- HUKUK-BOT V2.0 KALİTE KONTROL TESTİ ---
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

VECTOR_DB_PATH = "/content/drive/MyDrive/hukuk-bot/data/vector_db"
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

print("🔍 Veritabanı Kalite Kontrolü Başlıyor...")

if os.path.exists(VECTOR_DB_PATH):
    db = Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=embedding_model)

    # Test Sorusu
    soru = "Kasten yaralama" # Sadece kelime aratalım

    print(f"\n❓ Aranan: {soru}")
    docs = db.similarity_search(soru, k=3)

    print("\n📄 BULUNAN SONUÇLAR (Chunk Yapısı):")
    for i, doc in enumerate(docs):
        print(f"\n--- SONUÇ {i+1} ---")
        print(f"📂 Kaynak: {doc.metadata.get('source', 'Bilinmiyor')}")
        print(f"📝 İçerik (İlk 150 karakter):")
        print(f"➡️ {doc.page_content[:150]}...")
        # BURADA DİKKAT: "Madde X -" diye mi başlıyor? Yoksa cümlenin ortasından mı?

else:
    print("❌ Veritabanı henüz oluşmamış.")

🔍 Veritabanı Kalite Kontrolü Başlıyor...

❓ Aranan: Kasten yaralama

📄 BULUNAN SONUÇLAR (Chunk Yapısı):

--- SONUÇ 1 ---
📂 Kaynak: tck2.pdf
📝 İçerik (İlk 150 karakter):
➡️ Madde 86- 89) 
Basit bir t ıbbi müdahale ile giderilebilecek ölçüde hafif yaralanma: 
Maddenin 2. f
ıkrasında yer alan bu terim ceza itibar ı ile en h...

--- SONUÇ 2 ---
📂 Kaynak: tck.pdf
📝 İçerik (İlk 150 karakter):
➡️ Madde 86- 89) 
Basit bir t ıbbi müdahale ile giderilebilecek ölçüde hafif yaralanma: 
Maddenin 2. f
ıkrasında yer alan bu terim ceza itibar ı ile en h...

--- SONUÇ 3 ---
📂 Kaynak: 1.5.4721_removed2.pdf
📝 İçerik (İlk 150 karakter):
➡️ Madde 945-  Alacaklı, rehinli taşınırın kaybolm ası, yok olması veya değerinin azalması 
yüzünden meydana gelen zararlardan, bunların kendi kusuru olm...


In [ ]:
!pip install streamlit pyngrok localtunnel -q
print("✅ Arayüz araçları kuruldu.")


ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel
✅ Arayüz araçları kuruldu.


In [ ]:
%%writefile app.py
import streamlit as st
import os
import torch
from unsloth import FastLanguageModel
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# --- SAYFA AYARLARI ---
st.set_page_config(page_title="Hukuk-Bot v2.0", page_icon="⚖️", layout="wide")

st.title("⚖️ Hukuk-Bot v2.0")
st.markdown("Türk Hukuku Yapay Zeka Asistanı | **RAG & Llama-3 Destekli**")

# --- YAN MENÜ ---
with st.sidebar:
    st.header("Hakkında")
    st.info("Bu bot, PDF hukuk kitaplarını tarayarak sorularınızı cevaplar.")
    st.warning("⚠️ Yasal tavsiye değildir.")

# --- MODEL VE VERİTABANI YÜKLEME (CACHE) ---
# Streamlit her işlemde modeli baştan yüklemesin diye @st.cache_resource kullanıyoruz
@st.cache_resource
def load_resources():
    print("🧠 Model ve Veritabanı Yükleniyor...")

    # 1. Model
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model)

    # 2. Veritabanı
    VECTOR_DB_PATH = "/content/drive/MyDrive/hukuk-bot/data/vector_db"
    embedding_func = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    db = Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=embedding_func)

    return model, tokenizer, db

# Kaynakları yükle (İlk açılışta biraz bekletir)
try:
    model, tokenizer, db = load_resources()
    st.success("Sistem Hazır! 🚀")
except Exception as e:
    st.error(f"Sistem yüklenirken hata oluştu: {e}")
    st.stop()

# --- SOHBET GEÇMİŞİ (MEMORY) ---
if "messages" not in st.session_state:
    st.session_state.messages = []

# Geçmiş mesajları ekrana bas
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# --- KULLANICI GİRDİSİ ---
if prompt := st.chat_input("Hukuki sorunuzu yazın..."):
    # 1. Kullanıcı mesajını ekrana bas
    st.chat_message("user").markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    # 2. Cevap Üretiliyor animasyonu
    with st.spinner('Kanun kitapları taranıyor...'):
        try:
            # A. RAG Arama (Akıllı Bölümleme ile)
            docs = db.similarity_search(prompt, k=4)

            if docs:
                context = "\n\n".join([doc.page_content for doc in docs])
                kaynaklar = set([d.metadata.get('source', 'Bilinmiyor') for d in docs])

                # B. Prompt Hazırla
                system_msg = "Sen uzman bir Türk Hukuku asistanısın. Aşağıdaki KANUN MADDELERİNİ kullanarak soruyu cevapla. Cevabın profesyonel ve Türkçe olsun. Kendi yorumunu katma."
                user_msg = f"BAĞLAM:\n{context}\n\nSORU:\n{prompt}"

                messages_api = [
                    {"role": "system", "content": system_msg},
                    {"role": "user", "content": user_msg}
                ]

                # C. Modele Gönder
                input_ids = tokenizer.apply_chat_template(messages_api, add_generation_prompt=True, return_tensors="pt").to("cuda")

                outputs = model.generate(
                    input_ids,
                    max_new_tokens=512,
                    do_sample=True,
                    temperature=0.1,
                    pad_token_id=tokenizer.eos_token_id
                )

                response = outputs[0][input_ids.shape[-1]:]
                cevap_metni = tokenizer.decode(response, skip_special_tokens=True)

                # Kaynakları cevabın altına ekle
                kaynak_notu = "\n\n--- \n📚 **Kaynaklar:** " + ", ".join(kaynaklar)
                final_cevap = cevap_metni + kaynak_notu

            else:
                final_cevap = "⚠️ Üzgünüm, kanun kitaplarında bu konuyla ilgili bilgi bulamadım."

            # 3. Cevabı Ekrana Bas
            with st.chat_message("assistant"):
                st.markdown(final_cevap)
            st.session_state.messages.append({"role": "assistant", "content": final_cevap})

        except Exception as e:
            st.error(f"Bir hata oluştu: {e}")

Writing app.py


In [5]:
import os
import time
import subprocess
import re

# --- 1. TEMİZLİK VE KURULUM ---
print("🧹 Eski işlemler temizleniyor...")
!pkill -f streamlit
!pkill -f cloudflared
!rm -f nohup.out

print("🛠️ Gerekli kütüphaneler kuruluyor...")
!pip install streamlit langchain langchain-community langchain-huggingface langchain-chroma chromadb sentence-transformers unsloth -q

# --- 2. APP.PY DOSYASINI OLUŞTUR (RAM Dostu Sürüm) ---
print("📝 app.py dosyası oluşturuluyor...")
app_code = """
import streamlit as st
import os
import torch
import gc
from unsloth import FastLanguageModel
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

st.set_page_config(page_title="Hukuk-Bot v2.0 (Lite)", page_icon="⚖️")
st.title("⚖️ Hukuk-Bot v2.0 (Cloudflare)")
st.info("Türk Hukuku Yapay Zeka Asistanı | RAM Optimize Edilmiş Sürüm")

@st.cache_resource
def load_resources():
    # RAM Temizliği
    torch.cuda.empty_cache()
    gc.collect()

    print("🧠 Model Yükleniyor...")
    # LLM Modeli (GPU'da kalacak)
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
        max_seq_length = 1024, # RAM için kıstık
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model)

    print("📚 Veritabanı CPU'ya Yükleniyor...")
    VECTOR_DB_PATH = "/content/drive/MyDrive/hukuk-bot/data/vector_db"

    # ÖNEMLİ: Embedding'i CPU'ya alarak GPU'yu kurtarıyoruz
    embedding_func = HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        model_kwargs={'device': 'cpu'}
    )

    if os.path.exists(VECTOR_DB_PATH):
        db = Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=embedding_func)
    else:
        db = None
    return model, tokenizer, db

try:
    with st.spinner("Sistem hafifletilmiş modda açılıyor (1-2 dk)..."):
        model, tokenizer, db = load_resources()

    if db:
        st.success("Sistem Hazır! Sorunuzu sorun.")
    else:
        st.error("⚠️ Veritabanı bulunamadı! Drive yolunu kontrol edin.")

except Exception as e:
    st.error(f"Başlatma Hatası: {e}")

if "messages" not in st.session_state:
    st.session_state.messages = []

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("Hukuki sorunuzu yazın..."):
    # Her sorgudan önce RAM temizle
    torch.cuda.empty_cache()

    st.chat_message("user").markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    with st.spinner('Araştırılıyor...'):
        try:
            # k=3 ile daha az veri çekiyoruz
            docs = db.similarity_search(prompt, k=3)

            if docs:
                context = "\\n\\n".join([doc.page_content for doc in docs])
                # Context çok uzunsa kırp (RAM koruması)
                if len(context) > 2000: context = context[:2000]

                kaynaklar = set([d.metadata.get('source', 'Bilinmiyor') for d in docs])

                messages_api = [
                    {"role": "system", "content": "Sen Türk Hukuku uzmanısın. Kullanıcı hukuki soru sorarsa SADECE verilen [BAĞLAM] metnini kullan. Sohbet ediyorsa samimi ol."},
                    {"role": "user", "content": f"BAĞLAM:\\n{context}\\n\\nSORU:\\n{prompt}"}
                ]

                input_ids = tokenizer.apply_chat_template(messages_api, add_generation_prompt=True, return_tensors="pt").to("cuda")

                outputs = model.generate(
                    input_ids,
                    max_new_tokens=256,
                    temperature=0.3,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                )
                response = outputs[0][input_ids.shape[-1]:]
                cevap = tokenizer.decode(response, skip_special_tokens=True)

                final_cevap = cevap + "\\n\\n--- \\n📚 **Kaynak:** " + ", ".join(kaynaklar)
            else:
                final_cevap = "⚠️ Kaynaklarda bilgi bulunamadı."

            with st.chat_message("assistant"):
                st.markdown(final_cevap)
            st.session_state.messages.append({"role": "assistant", "content": final_cevap})

        except Exception as e:
            st.error(f"Hata: {e}")
            torch.cuda.empty_cache() # Hata olsa bile temizle
"""

with open("app.py", "w") as f:
    f.write(app_code)

# --- 3. STREAMLIT BAŞLAT ---
print("🚀 Streamlit başlatılıyor...")
get_ipython().system_raw('streamlit run app.py &')

# --- 4. CLOUDFLARE TÜNELİ (Token İstemez) ---
if not os.path.exists("cloudflared"):
    !wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared

print("🔗 Tünel açılıyor (Lütfen bekleyin)...")
# Logları 'nohup.out' dosyasına yazdırıyoruz ki linki oradan okuyabilelim
get_ipython().system_raw('./cloudflared tunnel --url http://localhost:8501 > nohup.out 2>&1 &')

# Linki Yakala
found_url = None
for i in range(20): # 40 saniye kadar bekle
    if os.path.exists("nohup.out"):
        with open("nohup.out", "r") as f:
            content = f.read()
            # Regex ile linki cımbızla çek
            match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', content)
            if match:
                found_url = match.group(0)
                break
    time.sleep(2)
    print(".", end="")

if found_url:
    print(f"\n\n✅ SİTE HAZIR! Tıkla: {found_url}")
    print("⚠️ UYARI: Siteye girince sağ üstte 'Running' yazıyorsa veya çark dönüyorsa BEKLE. Model yükleniyor (1-2 dk sürer).")
else:
    print("\n❌ Link bulunamadı. Lütfen hücreyi tekrar çalıştır.")

🧹 Eski işlemler temizleniyor...
🛠️ Gerekli kütüphaneler kuruluyor...
📝 app.py dosyası oluşturuluyor...
🚀 Streamlit başlatılıyor...
🔗 Tünel açılıyor (Lütfen bekleyin)...
..

✅ SİTE HAZIR! Tıkla: https://repairs-jersey-arena-hunt.trycloudflare.com
⚠️ UYARI: Siteye girince sağ üstte 'Running' yazıyorsa veya çark dönüyorsa BEKLE. Model yükleniyor (1-2 dk sürer).
